In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,5517
2,Huelva,Confirmados PDIA 14 días,2320
3,Huelva,Tasa PDIA 14 días,"454,24019516664936"
4,Huelva,Confirmados PDIA 7 días,1043
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,53
629,Municipio de Huelva sin especificar,Total Confirmados,127
630,Municipio de Huelva sin especificar,Curados,20


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  5517.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  1453.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 534 personas en los últimos 7 días 

Un positivo PCR cada 239 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,5517.0,1043.0,2320.0,510743.0,204.212295,454.240195,240.0
Huelva-Costa,2741.0,488.0,1106.0,288115.0,169.376811,383.874495,114.0
Huelva (capital),1453.0,269.0,601.0,143663.0,187.243758,418.340143,61.0
Condado-Campiña,1773.0,296.0,769.0,155057.0,190.897541,495.946652,61.0
Sierra de Huelva-Andévalo Central,877.0,206.0,361.0,67571.0,304.864513,534.252860,41.0
Gibraleón,250.0,72.0,163.0,12607.0,571.111287,1292.932498,30.0
Lepe,418.0,82.0,175.0,27431.0,298.931865,637.964347,29.0
Aracena,200.0,90.0,125.0,8107.0,1110.151721,1541.877390,19.0
Valverde del Camino,185.0,55.0,108.0,12820.0,429.017161,842.433697,12.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Escacena del Campo,80.0,13.0,53.0,2284.0,569.176883,2320.490368,2.0
Aracena,200.0,90.0,125.0,8107.0,1110.151721,1541.877390,19.0
Gibraleón,250.0,72.0,163.0,12607.0,571.111287,1292.932498,30.0
Villarrasa,66.0,2.0,28.0,2176.0,91.911765,1286.764706,0.0
Palma del Condado (La),255.0,44.0,104.0,10761.0,408.883933,966.452932,3.0
Valverde del Camino,185.0,55.0,108.0,12820.0,429.017161,842.433697,12.0
Manzanilla,37.0,6.0,17.0,2135.0,281.030445,796.252927,0.0
Villalba del Alcor,73.0,10.0,24.0,3338.0,299.580587,718.993409,3.0
Niebla,54.0,11.0,27.0,4117.0,267.184843,655.817343,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villarrasa,66.0,2.0,28.0,2176.0,91.911765,1286.764706,0.0,0.071429
San Juan del Puerto,97.0,1.0,11.0,9300.0,10.752688,118.279570,0.0,0.090909
Rosal de la Frontera,15.0,1.0,7.0,1697.0,58.927519,412.492634,0.0,0.142857
Escacena del Campo,80.0,13.0,53.0,2284.0,569.176883,2320.490368,2.0,0.245283
Cortegana,121.0,4.0,16.0,4666.0,85.726532,342.906129,0.0,0.250000
Almonte,285.0,41.0,142.0,24191.0,169.484519,586.995163,7.0,0.288732
Cartaya,124.0,15.0,48.0,19974.0,75.097627,240.312406,2.0,0.312500
Isla Cristina,148.0,15.0,47.0,21264.0,70.541761,221.030850,4.0,0.319149
Zarza-Perrunal (La),6.0,1.0,3.0,1267.0,78.926598,236.779795,0.0,0.333333
